In [194]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
import pandas as pd
import numpy as np
import geopandas as gpd

from bokeh.plotting import figure, show, save, ColumnDataSource
from bokeh.models import ColumnDataSource, LabelSet, Label,HoverTool,LinearColorMapper,ColumnDataSource
from bokeh.models import LinearInterpolator,  ColorBar,CDSView, GroupFilter
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap
from bokeh.models import LogColorMapper, LogTicker, ColorBar
from bokeh.layouts import row

In [195]:
output_notebook()

Loading BokehJS ...

In [196]:
output_file('Voronoi.html', title='Voronoi prices')

NameError: name 'output_file' is not defined

In [ ]:
voronoi_poly = gpd.read_file(r'D:/viz/trainlines/derived/New folder/derived_data.gpkg', layer='vor_cliped_single')

In [ ]:
voronoi_poly_wm = voronoi_poly.to_crs({'init': 'epsg:3857'})

In [ ]:
stations = gpd.read_file(r'D:/viz/trainlines/derived/New folder/derived_data.gpkg', layer='railstations')
#stations.head()

In [ ]:
stations_wm=stations.to_crs({'init': 'epsg:3857'})

In [212]:
sstations_wm = stations_wm.query('ttpe == "subway"')
rstations_wm = stations_wm.query('ttpe != "subway"')

In [213]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [214]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

In [215]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [216]:
# Calculate x and y coordinates of the line
voronoi_poly_wm['x'] = voronoi_poly_wm.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
voronoi_poly_wm['y'] = voronoi_poly_wm.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)


In [217]:
# Calculate x and y coordinates of the line
rstations_wm['x'] =rstations_wm.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
rstations_wm['y'] = rstations_wm.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)


sstations_wm['x'] = sstations_wm.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
sstations_wm['y'] = sstations_wm.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

C:\Users\ng93n\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ng93n\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ng93n\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [218]:
# Make a copy, drop the geometry column and create ColumnDataSource
vp = voronoi_poly_wm.drop('geometry', axis=1).copy()
vpsource = ColumnDataSource(vp)

In [219]:
# Make a copy, drop the geometry column and create ColumnDataSource
sst = sstations_wm.drop('geometry', axis=1).copy()
stsource = ColumnDataSource(sst)

rst = rstations_wm.drop('geometry', axis=1).copy()
rstsource = ColumnDataSource(rst)

In [220]:
p = figure(x_range=(-493900,-448223), y_range=(7511710,7547823),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)


TileRenderer(id='12235', ...)

In [221]:
p.patches('x', 'y', source=vpsource,
         fill_color= 'green', #{'field': 'pt_r_tt_ud', 'transform': color_mapper},
         fill_alpha=0, line_color="black", line_width=0.2)

GlyphRenderer(id='12242', ...)

In [226]:
stations = p.circle('x', 'y', size=6, source=rstsource,  line_color="#3288bd", line_width=1)

In [227]:
subway = p.circle('x', 'y', size=8, source=stsource,  line_color="orange", fill_color="white", line_width=1)

In [228]:
#removing grid lines

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#hiding axis
p.axis.visible = False

In [229]:
show(p)